In [1]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

2023-03-02 16:42:36.882056: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Define the intent labels
labels = ["效率", "良率", "時間","機台", "員工"]

# Define the training data
train_data = [
    ("今天工廠整場表現", "效率"),
    ("哪幾台產出最多良品", "良率"),
    ("有哪些機台異常？", "機台"),
    ("哪些工單會遲交", "時間"),
    ("請問 小王 今天表現如何", "員工"),
    ("請問 小明 這週表現如何", "員工")
]

# Tokenize the training data
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text for text, label in train_data])

# Convert the training data to sequences
train_sequences = tokenizer.texts_to_sequences([text for text, label in train_data])
print("[text for text, label in train_data]", [text for text, label in train_data])
print("train_sequences", train_sequences)

# Pad the sequences to a fixed length
max_length = max(len(seq) for seq in train_sequences)
train_sequences = pad_sequences(train_sequences, maxlen=max_length, padding='post')

# Convert the labels to one-hot encodings
label_encoder = {label: i for i, label in enumerate(labels)}
train_labels = [label_encoder[label] for text, label in train_data]
train_labels = tf.keras.utils.to_categorical(train_labels, num_classes=len(labels))

# Define the model architecture
model = tf.keras.Sequential()
model.add(layers.Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=32, input_length=max_length))
model.add(layers.Conv1D(64, 5, activation='relu', padding="same"))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(len(labels), activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(train_sequences, train_labels, epochs=100)

# Evaluate the model
test_data = [
    ("今天有 Gp122. 來的急出貨嗎？", "時間"),
    ("阿華工作進度如何", "員工"),
    ("機台 H122 怎麼沒在動 ", "機台"),
    ("事不是可以提高訂單量", "效率")
]

test_sequences = tokenizer.texts_to_sequences([text for text, label in test_data])
test_sequences = pad_sequences(test_sequences, maxlen=max_length, padding='post')
test_labels = [label_encoder[label] for text, label in test_data]
test_labels = tf.keras.utils.to_categorical(test_labels, num_classes=len(labels))

loss, accuracy = model.evaluate(test_sequences, test_labels)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")


[text for text, label in train_data] ['今天工廠整場表現', '哪幾台產出最多良品', '有哪些機台異常？', '哪些工單會遲交', '請問 小王 今天表現如何', '請問 小明 這週表現如何']
train_sequences [[2], [3], [4], [5], [1, 6, 7], [1, 8, 9]]
Epoch 1/100


2023-03-02 16:42:40.000483: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


1/1 [==============================] - 1s 1s/step - loss: 1.6102 - accuracy: 0.1667
Epoch 2/100
1/1 [==============================] - 0s 5ms/step - loss: 1.6064 - accuracy: 0.1667
Epoch 3/100
1/1 [==============================] - 0s 4ms/step - loss: 1.6028 - accuracy: 1.0000
Epoch 4/100
1/1 [==============================] - 0s 4ms/step - loss: 1.5988 - accuracy: 0.8333
Epoch 5/100
1/1 [==============================] - 0s 4ms/step - loss: 1.5942 - accuracy: 0.8333
Epoch 6/100
1/1 [==============================] - 0s 4ms/step - loss: 1.5892 - accuracy: 0.8333
Epoch 7/100
1/1 [==============================] - 0s 5ms/step - loss: 1.5835 - accuracy: 0.8333
Epoch 8/100
1/1 [==============================] - 0s 4ms/step - loss: 1.5771 - accuracy: 0.8333
Epoch 9/100
1/1 [==============================] - 0s 4ms/step - loss: 1.5703 - accuracy: 0.8333
Epoch 10/100
1/1 [==============================] - 0s 4ms/step - loss: 1.5625 - accuracy: 0.8333
Epoch 11/100
1/1 [========================

In [3]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 3, 32)             320       
                                                                 
 conv1d (Conv1D)             (None, 3, 64)             10304     
                                                                 
 global_max_pooling1d (Globa  (None, 64)               0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 128)               8320      
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                        

In [4]:
questions = "請問 陳先生表現如合？"
sequence_input = tokenizer.texts_to_sequences([questions])
padded_input = pad_sequences(sequence_input, maxlen=max_length , padding="post")
padded_input

array([[1, 0, 0]], dtype=int32)

In [5]:
result = model.predict(padded_input)

1/1 [==============================] - 0s 99ms/step


In [6]:
result

array([[0.00700853, 0.17302732, 0.5845128 , 0.21982652, 0.01562475]],
      dtype=float32)

In [7]:
predicted_probs = model.predict(padded_input)[0]
predicted_label_index = np.argmax(result)


if predicted_probs[predicted_label_index] < 0.9:
    print("老賽！")
else:
    predicted_label = labels[predicted_label_index]
    print("predicted_label", predicted_label)

1/1 [==============================] - 0s 18ms/step
老賽！
